In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from PIL import Image
import cv2
import joblib

#감마보정

In [2]:
def gamma_correction(image, gamma=1.0):
    inv_gamma = 1/ gamma
    correction_img = np.uint8(((image / 255) ** inv_gamma) * 255)
    return correction_img

#이미지 크롭 예제

In [3]:
def process_image_with_mouse_callback(image_path):
    # 초기화된 변수
    src_quad = np.zeros((4, 2), dtype=np.int32)  # Array to store the clicked coordinates
    count = 0  # Counter for clicked points
    cropped_img = None

    # Load the image
    src = cv2.imread(image_path)
    if src is None:
        print("Image load failed!")
        return None

    # Resize image while maintaining aspect ratio
    target_width = 640
    target_height = int((src.shape[0] * target_width) / src.shape[1])
    resized_src = cv2.resize(src, (target_width, target_height))

    # 마우스 콜백 함수 정의
    def on_mouse(event, x, y, flags, param):
        nonlocal count, src_quad, cropped_img  # 함수 내부에서 외부 변수를 사용하도록 설정

        if event == cv2.EVENT_LBUTTONDOWN and count < 4:
            # Store the clicked point in src_quad
            src_quad[count] = [x, y]
            count += 1

            # Display a circle at the clicked point
            temp_image = resized_src.copy()
            for i in range(count):
                cv2.circle(temp_image, (int(src_quad[i][0]), int(src_quad[i][1])), 5, (0, 0, 255), -1)

            cv2.imshow("src", temp_image)

            # If 4 points are clicked, perform the cropping
            if count == 4:
                # Calculate bounding box (rectangle) for the selected points
                x_min, y_min = np.min(src_quad, axis=0)
                x_max, y_max = np.max(src_quad, axis=0)

                # Crop the selected area
                cropped_img = resized_src[y_min:y_max, x_min:x_max]

                # Show the cropped image
                cv2.imshow("cropped", cropped_img)

    # Create a window and set mouse callback
    cv2.namedWindow("src")
    cv2.setMouseCallback("src", on_mouse)

    # Display resized image and wait for user to click points
    cv2.imshow("src", resized_src)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Return the cropped image if available
    return cropped_img

# RGB to CMYK 변환 함수 (백분율로 출력)
def rgb_to_cmyk(img):
    bgr = img.astype(np.float64) / 255.0
    k = 1 - np.max(bgr, axis=2)
    mask = (1 - k) > 0
    c = np.where(mask, (1 - bgr[..., 2] - k) / (1 - k), 0)
    m = np.where(mask, (1 - bgr[..., 1] - k) / (1 - k), 0)
    y = np.where(mask, (1 - bgr[..., 0] - k) / (1 - k), 0)
    cmyk = (np.dstack((c, m, y, k)) * 100).astype(np.float64)
    return cmyk

##수정중(이미지 크롭 매개변수 주소 -> 변수)

In [4]:
import numpy as np
import cv2

def process_image_with_mouse_callback_temp(image):
    # 초기화된 변수
    src_quad = np.zeros((4, 2), dtype=np.int32)  # Array to store the clicked coordinates
    count = 0  # Counter for clicked points
    cropped_img = None

    # 이미지가 None인지 확인
    if image is None:
        print("Image load failed!")
        return None

    # Resize image while maintaining aspect ratio
    target_width = 640
    target_height = int((image.shape[0] * target_width) / image.shape[1])
    resized_src = cv2.resize(image, (target_width, target_height))

    # 마우스 콜백 함수 정의
    def on_mouse(event, x, y, flags, param):
        nonlocal count, src_quad, cropped_img  # 함수 내부에서 외부 변수를 사용하도록 설정

        if event == cv2.EVENT_LBUTTONDOWN and count < 4:
            # Store the clicked point in src_quad
            src_quad[count] = [x, y]
            count += 1

            # Display a circle at the clicked point
            temp_image = resized_src.copy()
            for i in range(count):
                cv2.circle(temp_image, (int(src_quad[i][0]), int(src_quad[i][1])), 5, (0, 0, 255), -1)

            cv2.imshow("src", temp_image)

            # If 4 points are clicked, perform the cropping
            if count == 4:
                # Calculate bounding box (rectangle) for the selected points
                x_min, y_min = np.min(src_quad, axis=0)
                x_max, y_max = np.max(src_quad, axis=0)

                # Crop the selected area
                cropped_img = resized_src[y_min:y_max, x_min:x_max]

                # Show the cropped image
                cv2.imshow("cropped", cropped_img)

    # Create a window and set mouse callback
    cv2.namedWindow("src")
    cv2.setMouseCallback("src", on_mouse)

    # Display resized image and wait for user to click points
    cv2.imshow("src", resized_src)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Return the cropped image if available
    return cropped_img

# RGB to CMYK 변환 함수 (백분율로 출력)
def rgb_to_cmyk(img):
    bgr = img.astype(np.float64) / 255.0
    k = 1 - np.max(bgr, axis=2)
    mask = (1 - k) > 0
    c = np.where(mask, (1 - bgr[..., 2] - k) / (1 - k), 0)
    m = np.where(mask, (1 - bgr[..., 1] - k) / (1 - k), 0)
    y = np.where(mask, (1 - bgr[..., 0] - k) / (1 - k), 0)
    cmyk = (np.dstack((c, m, y, k)) * 100).astype(np.float64)
    return cmyk


#데이터프레임

In [5]:
df = pd.read_csv("/Users/ansejun/Desktop/Python3/Test/cmyk.csv")  # 데이터프레임 링크
df

,pH,C,M,Y,K
0,1.00,0.0000,69.0000,77.00,2.0000
1,1.01,0.0000,68.8400,76.97,2.0000
2,1.02,0.0000,68.6800,76.94,2.0000
3,1.03,0.0000,68.5200,76.91,2.0000
4,1.04,0.0000,68.3600,76.88,2.0000
...,...,...,...,...,...
996,10.96,13.5940,33.1972,0.00,51.0740
997,10.97,13.5955,33.2979,0.00,51.0505
998,10.98,13.5970,33.3986,0.00,51.0270
999,10.99,13.5985,33.4993,0.00,51.0035


#RGB to CMYK

In [6]:
# RGB to CMYK 변환 함수 (백분율로 출력)
def rgb_to_cmyk(img):
    # BGR을 0~1 범위로 정규화
    bgr = img.astype(np.float64) / 255.0

    # K 채널 계산
    k = 1 - np.max(bgr, axis=2)

    # K가 1이 아닐 때만 CMY 값 계산
    mask = (1 - k) > 0
    c = np.where(mask, (1 - bgr[..., 2] - k) / (1 - k), 0)
    m = np.where(mask, (1 - bgr[..., 1] - k) / (1 - k), 0)
    y = np.where(mask, (1 - bgr[..., 0] - k) / (1 - k), 0)

    # CMYK를 백분율로 변환 (0~100)
    cmyk = (np.dstack((c, m, y, k)) * 100).astype(np.float64)
    return cmyk

In [7]:
def measure_ph(img):
    if img is None:
        print("이미지를 열 수 없습니다. 경로를 확인하세요.")
        return None

    # CMYK 변환 (0~100 백분율로 변환된 CMYK 이미지)
    cmyk_image = rgb_to_cmyk(img)

    # CMYK 값을 튜플 형태로 가져와 빈도수 계산
    pixels = [tuple(pixel) for pixel in cmyk_image.reshape(-1, 4)]  # 각 픽셀을 CMYK 튜플로 변환

    color_counts = {}
    for pixel in pixels:
        if pixel in color_counts:
            color_counts[pixel] += 1
        else:
            color_counts[pixel] = 1

    # 가장 많은 빈도수를 가진 색상 값 찾기
    most_common_color = max(color_counts, key=color_counts.get)
    print("가장 많은 픽셀의 CMYK 색상 값 (백분율):", most_common_color)

    # CMYK 색상값 반환
    return most_common_color


#모델학습

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
X = df[['C', 'M', 'Y', 'K']]
y = df['pH']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
rf_model = RandomForestRegressor(n_estimators=150, random_state=42)
rf_model.fit(X_train, y_train)

NameError: name 'RandomForestRegressor' is not defined

#모델 저장

In [ ]:
# 학습된 모델 저장
joblib.dump(rf_model, 'random_forest_model.joblib')

#모델 불러오기

In [17]:
# 모델 불러오기
rf_model = joblib.load('random_forest_model.joblib')


In [18]:
# 100개 : 2.9()
# 150개 : 2.7(0.9)
# 200개 : 2.9(0.9)
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("평균 제곱 오차 (MSE):", mse)
print("R² 점수:", r2)

평균 제곱 오차 (MSE): 2.6814704256490426e-05
R² 점수: 0.9999967369572748


In [124]:
# 100개 : 2.9()
# 150개 : 2.7(0.9)
# 200개 : 2.9(0.9)
y_pred = test_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("평균 제곱 오차 (MSE):", mse)
print("R² 점수:", r2)

평균 제곱 오차 (MSE): 2.386695411828625e-05
R² 점수: 0.9999970956647418


#TEST

In [ ]:
temp = rgb_to_cmyk(img)
print(temp)

In [ ]:
# 지정 CMYK값의 pH 계산
new_data = [[106, 92, 190, 0]]

predicted_label = rf_model.predict(new_data)

print("예측된 label 값:", predicted_label[0])

In [115]:
# 이미지 CMYK값의 pH 계산
# ph1, ph2, ph3, ph5
img = cv2.imread('/Users/ansejun/Desktop/Python3/Test/ph5.png') # 샘플 이미지 주소

# CMYK 색상 값 받아오기
cmyk_value = measure_ph(img)
if cmyk_value:
    # 랜덤 포레스트 모델에 입력할 데이터 준비
    new_data = [[cmyk_value[0], cmyk_value[1], cmyk_value[2], cmyk_value[3]]]

    # 예측 수행
    predicted_label = rf_model.predict(new_data)
    print("예측된 label 값:", predicted_label[0])

가장 많은 픽셀의 CMYK 색상 값 (백분율): (3.684210526315791, 0.0, 59.473684210526315, 25.49019607843137)
예측된 label 값: 5.404333333333336


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [ ]:
cropped_image = process_image_with_mouse_callback("/Users/ansejun/Desktop/Python3/Test/123.jpeg")

if cropped_image is not None:
    cmyk_value = measure_ph(cropped_image)
    if cmyk_value is not None:
        # 랜덤 포레스트 모델에 입력할 데이터 준비
        new_data = [cmyk_value[:4]]  # CMYK 값을 2D 배열로 포장하여 입력

        # 예측 수행 (모델이 이미 학습된 상태여야 함)
        predicted_label = rf_model.predict(new_data)
        print("예측된 label 값:", predicted_label[0])

In [116]:
row_img = cv2.imread('/Users/ansejun/Desktop/Python3/Test/ph5.png')
img = gamma_correction(row_img, gamma=1.1)

# CMYK 색상 값 받아오기
cmyk_value = measure_ph(img)
if cmyk_value:
    # 랜덤 포레스트 모델에 입력할 데이터 준비
    new_data = [[cmyk_value[0], cmyk_value[1], cmyk_value[2], cmyk_value[3]]]

    # 예측 수행
    predicted_label = rf_model.predict(new_data)
    print("예측된 label 값:", predicted_label[0])

가장 많은 픽셀의 CMYK 색상 값 (백분율): (3.589743589743577, 0.0, 56.41025641025642, 23.529411764705888)
예측된 label 값: 5.4006


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


#보강학습 후 모델 테스트(test_random_forest_model)

In [117]:
# 모델 불러오기
test_model = joblib.load('test_random_forest_model.joblib')


In [153]:
row_img = cv2.imread('/Users/ansejun/Desktop/Python3/Test/ph4.png')
img = gamma_correction(row_img, gamma=1.1)

# CMYK 색상 값 받아오기
cmyk_value = measure_ph(img)
if cmyk_value:
    # 랜덤 포레스트 모델에 입력할 데이터 준비
    new_data = [[cmyk_value[0], cmyk_value[1], cmyk_value[2], cmyk_value[3]]]

    # 예측 수행
    predicted_label = test_model.predict(new_data)
    print("예측된 label 값:", predicted_label[0])

가장 많은 픽셀의 CMYK 색상 값 (백분율): (0.0, 16.894977168949765, 61.18721461187215, 14.117647058823534)
예측된 label 값: 4.352466666666671


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
